In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS
import pathlib
from pathlib import Path
from shapely import wkt
from tqdm import tqdm

import math
import codecs
import osm2geojson
from shapely import wkt

import gzip
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import xml.etree.ElementTree as ET

import seaborn as sns

# set the working directory
BASE_DIR = Path.cwd()

# auto reloading
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
dfSWITRS = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","CollisionRecords.txt"))
dfTIMSCity = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","TIMS_Crashes_city.csv"))
dfTIMSStateRoutes = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","TIMS_Crahes_StateRoutes.csv"))

# SWITRS database
dfSWITRS = dfSWITRS[dfSWITRS["ACCIDENT_YEAR"].isin([2010,2016])]
# TIMS database
dfTIMSCity = dfTIMSCity[dfTIMSCity["ACCIDENT_YEAR"].isin([2010,2016])]
dfTIMSStateRoutes = dfTIMSStateRoutes[dfTIMSStateRoutes["ACCIDENT_YEAR"].isin([2010,2016])]

c:\users\goyal\.virtualenvs\overpass_turbo-zwxzihl_\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\goyal\.virtualenvs\overpass_turbo-zwxzihl_\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (7,14,17,26,29,30,33,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
frames = [dfTIMSCity,dfTIMSStateRoutes]

result = pd.concat(frames)

result['POINT_X']= np.where(result['POINT_X'].isnull(), result['LONGITUDE'], result['POINT_X'])
result['POINT_Y']= np.where(result['POINT_Y'].isnull(), result['LATITUDE'], result['POINT_Y'])

result = result.drop_duplicates(subset=['CASE_ID'],keep="first")
result.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","TIMS_all.csv"))

frames = [result,dfSWITRS]
result_2 = pd.concat(frames)
result_2['POINT_X']= np.where(result_2['POINT_X'].isnull(), result_2['LONGITUDE'], result_2['POINT_X'])
result_2['POINT_Y']= np.where(result_2['POINT_Y'].isnull(), result_2['LATITUDE'], result_2['POINT_Y'])

result_2['POINT_X']= np.where(result_2['POINT_X']>0, -abs(result_2['POINT_X']), result_2['POINT_X'])

result_3 = result_2.drop_duplicates(subset=['CASE_ID'])
result_3.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Switrs_TIMS_all.csv"))

In [4]:
# Reverse geocode

In [5]:
# read the exported file and also the actual data from SWITRS
BASE_DIR = Path.cwd()
dfIntersect_Rds = pd.read_csv(BASE_DIR.parent.joinpath("Exported_files","Intersecting_Rds.csv"))

dfIntersect_Rds['geometry'] = dfIntersect_Rds['geometry'].apply(wkt.loads)
dfIntersect_Rds = gpd.GeoDataFrame(dfIntersect_Rds, crs='epsg:4326')
col = ["Unnamed: 0", "index","layer","path"]
dfIntersect_Rds = dfIntersect_Rds.drop(col,axis=1).copy()

dfSWITRS = pd.read_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Switrs_TIMS_all.csv"))

c:\users\goyal\.virtualenvs\overpass_turbo-zwxzihl_\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (3,5,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
def cleanStreetName(streetName):
    newStreetName = streetName.strip()
    corrections = {"TWELFTH":"12TH", 
                   "ELEVENTH":"11TH",
                   "TENTH":"10TH",
                   "NINTH":"9TH",
                   "EIGHTH":"8TH",
                   "SEVENTH":"7TH",
                   "SIXTH":"6TH",
                   "FIFTH":"5TH",
                   "FOURTH":"4TH",
                   "THIRD":"3RD",
                   "SECOND":"2ND",
                   "FIRST":"1ST",
                   "O'FARRELL":"O FARRELL",
                   "3RDREET":"3RD",
                   "EMBARCADERO/KING":"THE EMBARCADERO",
                   "VAN NESSNUE":"VAN NESS",
                   "3RD #3":"3RD",
                   "BAYSHORE #3":"BAYSHORE",
                   "09TH":"9TH",
                   "08TH":"8TH",
                   "07TH":"7TH",
                   "06TH":"6TH",
                   "05TH":"5TH",
                   "04TH":"4TH",
                   "03RD":"3RD",
                   "02ND":"2ND",
                   "01ST":"1ST",
                  }

    itemsToRemove = [" STREETS",
                     " STS.",
                     " STS",
                     " ST.",
                     " ST",
                     " Street",
                     " RD.",
                     " RD",
                     " Road",
                     " AVE.",
                     " AVES",
                     " AVE",
                     " AV",
                     " Avenue",
                     " BLVD.",
                     " BLVD",
                     " BL",                     
                     " Boulevard",
                     " MASTER:",
                     " DR.",
                     " Drive",
                     " WY",
                     " Way",
                     " CT",
                     " TERR",
                     " Terrace",
                     " HWY"]
          
    for wrongName, rightName in corrections.items():
        if wrongName in streetName:
            newStreetName = streetName.replace(wrongName, rightName)
        if streetName == 'EMBARCADERO':
            newStreetName = "THE EMBARCADERO"
        if streetName.endswith(" DR"):
            newStreetName = streetName[:-3]
        if streetName.endswith(" AV"):
            newStreetName = streetName[:-3]
        if " TO " in streetName:
            cutOff = streetName.find(" TO ")
            newStreetName = streetName[:cutOff]            
    
    for item in itemsToRemove:
        if item in newStreetName:
            newStreetName = newStreetName.replace(item, "")
    
    return newStreetName.strip()


dfSWITRS.columns = dfSWITRS.columns.str.rstrip('_x')  # strip suffix at the right end only.
dfSWITRS["PRIMARY_RD_2"] = dfSWITRS.apply(lambda x: cleanStreetName(x["PRIMARY_RD"]), axis=1)
dfSWITRS["SECONDARY_RD_2"] = dfSWITRS.apply(lambda x: cleanStreetName(x["SECONDARY_RD"]), axis=1)

# dfSWITRS['Rd_Names'] = dfSWITRS[['PRIMARY_RD_2', 'SECONDARY_RD_2']].apply(lambda x: ' '.join(x), axis=1)
dfSWITRS['Rd_Names'] = dfSWITRS[['PRIMARY_RD_2', 'SECONDARY_RD_2']].apply(lambda x: ','.join(set(x.unique())), axis=1)
# remove first character from the values if it starts with ","
dfSWITRS['Rd_Names'] = dfSWITRS['Rd_Names'].apply(lambda x : x[1:] if x.startswith(",") else x)
# lowercase
dfSWITRS = dfSWITRS.applymap(lambda s:s.lower() if type(s) == str else s)

dfIntersect_Rds=dfIntersect_Rds.fillna("").copy()

dfIntersect_Rds["Link_ID_0_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_0"]), axis=1)
dfIntersect_Rds["Link_ID_1_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_1"]), axis=1)
dfIntersect_Rds["Link_ID_2_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_2"]), axis=1)
dfIntersect_Rds["Link_ID_3_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_3"]), axis=1)
dfIntersect_Rds["Link_ID_4_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_4"]), axis=1)
dfIntersect_Rds["Link_ID_5_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_5"]), axis=1)
dfIntersect_Rds["Link_ID_6_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_6"]), axis=1)
dfIntersect_Rds["Link_ID_7_2"] = dfIntersect_Rds.apply(lambda x: cleanStreetName(x["Link_ID_7"]), axis=1)

dfIntersect_Rds['Link_Names'] = dfIntersect_Rds[['Link_ID_0_2', 'Link_ID_1_2','Link_ID_2_2', 'Link_ID_3_2','Link_ID_4_2', 'Link_ID_5_2','Link_ID_6_2', 'Link_ID_7_2']].apply(lambda x: ','.join(set(x.unique())), axis=1)

# remove first character from the values if it starts with ","
dfIntersect_Rds['Link_Names'] = dfIntersect_Rds['Link_Names'].apply(lambda x : x[1:] if x.startswith(",") else x)

# lowercase
dfIntersect_Rds = dfIntersect_Rds.applymap(lambda s:s.lower() if type(s) == str else s)

dfIntersect_Rds["LatLong"] = dfIntersect_Rds["lat"].astype(str) + " " + dfIntersect_Rds["lon"].astype(str)
dfIntersect_Rds.to_csv(BASE_DIR.parent.joinpath("Exported_Files","Intersecting_Rds_2.csv"))

# get unique street intersections and their latitude and longtitude
dfunique = dfIntersect_Rds[['Link_Names', 'LatLong']]
# get unique street intersections and their latitude and longtitude
dfunique.drop_duplicates(['Link_Names'])
mydict = dict(zip(dfunique.Link_Names, dfunique.LatLong.astype(str)))

In [7]:
from shapely.geometry import Point

def getDerivedLatLngPosition(lat,long, dist, bearing):
    latitude = float(lat)
    longitude = float(long)
    
    DegreesToRadians = math.pi/180.0
    RadiansToDegrees = 180.0/math.pi 
    EarthRadius = 6378137.0 # Radius of Earth in meters

    latA =  latitude * DegreesToRadians
    longA = longitude * DegreesToRadians
    angularDistance = dist/EarthRadius
    trueCourse = bearing * DegreesToRadians
    
    lat =  math.asin(math.sin(latA) * math.cos(angularDistance) + math.cos(latA) * math.sin(angularDistance) * math.cos(trueCourse))
    
    dlon = math.atan2(math.sin(trueCourse) * math.sin(angularDistance) * math.cos(latA), math.cos(angularDistance) - math.sin(latA) * math.sin(lat))
    lon = ((longA + dlon + math.pi) % (math.pi*2)) - math.pi
    
    return Point(round(lon * RadiansToDegrees,5),round(lat * RadiansToDegrees,5))

# Switcher is dictionary data type here
def get_direction(argument):
    switcher = {
        "N": 0,
        "W": -90,
        "E": +90,
        "S": -180
    }  
    # get() method of dictionary data type returns value of passed argument if it is present in dictionary;
    # otherwise second argument will be assigned as default value of passed argument
    return switcher.get(argument, 0)

In [8]:
def latlongsearch(row):
    primary_rd = row["PRIMARY_RD_2"]
    secondary_rd = row["SECONDARY_RD_2"] 
    _dict = mydict.copy()
    for key, value in _dict.items():
        if (primary_rd in key) and (secondary_rd in key):
            lat = value.split(" ")[0]
            long = value.split(" ")[1]
            dist = 0 if (row["DISTANCE"]<= 0) else (row["DISTANCE"]*0.3048/3.2808)
#             print(row["DIRECTION"])
            bearing = get_direction(row["DIRECTION"].upper())
            return getDerivedLatLngPosition(lat,long, dist, bearing)
    return None         
dfSWITRS["DIRECTION"]=dfSWITRS["DIRECTION"].fillna('')
dfSWITRS["ref_latlong"] = dfSWITRS.apply(lambda row: latlongsearch(row), axis=1)

dfSWITRS.to_csv(BASE_DIR.parent.joinpath("Exported_Files","Road_Crashes_SWITRS_TIMS_matched_output.csv"))
dfSWITRS.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_SWITRS_TIMS_matched_output.csv"))

def fill_null_latlong(row):
    if pd.isnull(row["ref_latlong"]):
        if pd.notna(row["POINT_X"]):
            return Point(round(row["POINT_X"],5),round(row["POINT_Y"],5))
    else:
        return row["ref_latlong"]
dfSWITRS["ref_latlong"] = dfSWITRS.apply(lambda row: fill_null_latlong(row), axis=1)

dfSWITRS.to_csv(BASE_DIR.parent.joinpath("Data","02Oct2021","Road_Crashes_SWITRS_TIMS_matched_output_v2.csv"))
dfSWITRS.to_csv(BASE_DIR.parent.joinpath("Exported_Files","Road_Crashes_SWITRS_TIMS_matched_output.csv"))

In [9]:
####### for more detailed and split analysis please refer to 00.1 - SWITRS.ipynb file ###########

In [ ]:
# read 2010 file
dfSFRd = gpd.read_file(BASE_DIR.parent.joinpath("Data","san-francisco_california.geojsonl"))